# Computing Partial Derivatives for Neural Networks

### Setup

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tensorflow.contrib.layers as layers

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8) # set default size of plots
plt.rcParams['agg.path.chunksize'] = 10000
%config InlineBackend.figure_format = 'retina'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

sns.set_style("whitegrid")
sns.set_palette("bright")

### Helper methods

In [ ]:
def model(x, reuse=False):
    """
    2 Hidden Layer (5 x 5) neural network with RELU activation.
    """
    fc1 = layers.fully_connected(x, 5, activation_fn=tf.nn.relu, scope='fc1', reuse=reuse)
    fc2 = layers.fully_connected(fc1, 5, activation_fn=tf.nn.relu, scope='fc2', reuse=reuse)
    y = layers.fully_connected(fc2, 1, activation_fn=None, scope='fc3', reuse=reuse)
    return y

def preprocess(X):
    """
    Scales and shifts X to have 0 mean and unit variance.
    """
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X, scaler

def get_data(df, X_vars, y_var, train_test_split=1.0):
    """
    Gets normalized features and the response variable from the dataframe.
    
    Args:
      - df: Pandas dataframe containing the input data.
      - X_vars: List of column names for the input matrix.
      - y_var: Column name for the output matrix.
      - train_test_split: Percentage of data to go in train.
    
    Returns:
        - X: Normalized input matrix containing X_vars.
        - y: Array containing the response variable.
        - X_scaler: Module used to normalize X.
    """
    # Filter out rows where one of the input variables and/or response variable is NaN.
    df = df[X_vars + [y_var]].dropna()
    df = df.reindex(np.random.permutation(df.index))
    df_train = df.iloc[:int(df.shape[0] * train_test_split)]
    df_test = df.iloc[int(df.shape[0] * train_test_split):]
    X_train, X_scaler = preprocess(df_train[X_vars])
    X_test = X_scaler.transform(df_test[X_vars])
    y_train = df_train[y_var]
    y_test = df_test[y_var]
    return X_train, y_train, X_test, y_test, X_scaler

### Read the data

In [ ]:
US_Var_df = pd.read_csv("biomet_data/AMF_US-Var_BASE_HH_9-1.csv", comment = "#").replace("-9999", np.nan)
US_Ton_df = pd.read_csv("biomet_data/AMF_US-Ton_BASE_HH_9-1.csv", comment = "#").replace("-9999", np.nan)
print "Var Data: ", US_Var_df.shape
print "Tonzi Data: ", US_Ton_df.shape
print "Columns: ", US_Var_df.columns

### Display features and response variable

In [ ]:
X_vars, y_var, train_test_split = ['TA', 'SW_IN', 'RH'], 'NETRAD', 0.8
X_train, y_train, X_val, y_val, X_scaler = get_data(US_Var_df, X_vars, y_var, train_test_split)
print "X_train: ", X_train.shape
if X_train.shape[1] == 1:
    plt.scatter(X_scaler.inverse_transform(X_train), y_train)
    plt.xlabel(X_vars[0])
    plt.ylabel(y_var)
    plt.show()
    plt.xlabel(X_vars[0])
    sns.kdeplot(X_scaler.inverse_transform(X_train[:20000, 0]), y_train[:20000], cmap="Blues", shade=True, shade_lowest=False)
elif X_train.shape[1] == 2:
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
#     ax.xlim([-20,20])
    ax.scatter(X_train[:, 0], X_train[:, 1], y_train)
else:
    print "Cannot plot data when in {0}-D!".format(X_train.shape[1] + 1)

In [ ]:
X_uniform = np.mgrid[-2:2.01:0.08, -2:2.01:0.08, -2:2.01:0.08].reshape(3,-1).T
print X_uniform.shape

In [ ]:
X_gaussian = np.random.multivariate_normal([0, 0, 0], np.diag([1, 1, 1]), 50000)

### Train Neural Network, Compute Partials, Plot Result

Using a 80/20 split on the training data, I trained a 5x5 2 hidden layer neural network with a batch size of 256. I then computed the predictions and partial derivatives w.r.t. input on the validation set. As you may observe, the neural network tries to predict the mean of the response distribution at any given x. For the currently chosen parameters, this results in a negative almost straight line. The partial derivatives as a result are constant for many segments of x.

In [ ]:
# Split the data into batches
batch_size = 256
num_batches = X_train.shape[0] / batch_size
X_batches = np.array_split(X_train, num_batches)
y_batches = np.array_split(y_train.values.reshape(-1, 1), num_batches)

# Placeholders for the 
X = tf.placeholder(tf.float32, shape=[None, X_train.shape[1]])
y = tf.placeholder(tf.float32, shape=[None, 1])
y_pred = model(X, reuse=False)

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y_pred - y))
optimizer = tf.train.GradientDescentOptimizer(0.00001)
train = optimizer.minimize(loss)

# Compute partial derivative of predictions w.r.t. input.
partial_x = tf.gradients(y_pred, X)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

with tf.Graph().as_default(), sess.as_default():
    for epoch in xrange(30):
        for i in xrange(num_batches):
            # Train on batch i.
            _, batch_loss = sess.run([train, loss], feed_dict={X: X_batches[i], y: y_batches[i]})
            if i % 500 == 0:
                print "Epoch: ", epoch, "Batch: ", i, "MSE: ", batch_loss
    
    # Compute test predictions and partial derivatives w.r.t. input.
    preds, partials = sess.run([y_pred, partial_x], feed_dict={X: X_val})
    # Compute test predictions and partial derivatives w.r.t. input.
    preds, partials = sess.run([y_pred, partial_x], feed_dict={X: X_val})
    uniform_preds, uniform_partials = sess.run([y_pred, partial_x], feed_dict={X: X_uniform})
    gauss_preds, gauss_partials = sess.run([y_pred, partial_x], feed_dict={X: X_gaussian})

In [ ]:
# Plot the first 20000 samples for KDE and 1000 for preds, already randomly shuffled, to save performance.
plt.title("NETRAD vs. TA")
sns.kdeplot(X_scaler.inverse_transform(X_val)[:20000, 0], y_val[:20000], cmap="Blues", shade=True, shade_lowest=False)
plt.scatter(X_scaler.inverse_transform(X_val)[:1000, 0], preds[:1000, 0], c='r')
plt.show()
plt.title("NETRAD vs. SW_IN")
sns.kdeplot(X_scaler.inverse_transform(X_val)[:20000, 1], y_val[:20000], cmap="Blues", shade=True, shade_lowest=False)
plt.scatter(X_scaler.inverse_transform(X_val)[:1000, 1], preds[:1000, 0], c='r')
plt.show()
plt.title("NETRAD vs. RH")
sns.kdeplot(X_scaler.inverse_transform(X_val)[:20000, 2], y_val[:20000], cmap="Blues", shade=True, shade_lowest=False)
plt.scatter(X_scaler.inverse_transform(X_val)[:1000, 2], preds[:1000, 0], c='r')
plt.show()

In [ ]:
# plt.scatter(X_scaler.inverse_transform(X_val)[:20000, 0], X_scaler.inverse_transform(X_val)[:20000, 1], c='b')
# plt.scatter(X_scaler.inverse_transform(X_val)[:20000, 0], X_scaler.inverse_transform(X_val)[:20000, 1], c='b')

sns.kdeplot(X_scaler.inverse_transform(X_val)[:20000, 0], X_scaler.inverse_transform(X_val)[:20000, 1], cmap="Blues", shade=True, shade_lowest=False)




In [ ]:
plt.title("Partials of NETRAD w.r.t. TA")
plt.scatter(X_scaler.inverse_transform(X_val)[:, 0], partials[0][:, 0])

In [ ]:
plt.title("Partials of NETRAD w.r.t. SW_IN")
plt.scatter(X_scaler.inverse_transform(X_val)[:, 1], partials[0][:, 1])

In [ ]:
plt.title("Partials of NETRAD w.r.t. RH")
plt.scatter(X_scaler.inverse_transform(X_val)[:, 2], partials[0][:, 2])

In [ ]:
plt.title("Predictions of NETRAD w.r.t. TA")
plt.scatter(X_scaler.inverse_transform(X_val)[:20000, 0], preds[:20000, 0], c='r')

In [ ]:
plt.title("Predictions of NETRAD w.r.t. SW_IN")
plt.scatter(X_scaler.inverse_transform(X_val)[:20000, 1], preds[:20000, 0], c='r')

In [ ]:
plt.title("Predictions of NETRAD w.r.t. RH")
plt.scatter(X_scaler.inverse_transform(X_val)[:20000, 2], preds[:20000, 0], c='r')

In [ ]:
plt.title("Prediction vs. Actual")
plt.scatter(y_val[:20000], preds[:20000])
plt.plot([-200, 800], [-200, 800])

## Uniform Predictions/Partials

In [ ]:
plt.title("Predictions of NETRAD w.r.t. TA (Uniform)")
plt.scatter(X_scaler.inverse_transform(X_uniform)[:, 0], uniform_preds[:, 0], c='r')

In [ ]:
plt.title("Partials of NETRAD w.r.t. TA")
plt.scatter(X_scaler.inverse_transform(X_uniform)[:, 0], uniform_partials[0][:, 0])

In [ ]:
plt.title("Predictions of NETRAD w.r.t. SW_IN")
plt.scatter(X_scaler.inverse_transform(X_uniform)[:, 1], uniform_preds[:, 0], c='r')

In [ ]:
plt.title("Partials of NETRAD w.r.t. SW_IN")
plt.scatter(X_scaler.inverse_transform(X_uniform)[:, 1], uniform_partials[0][:, 1])

In [ ]:
plt.title("Predictions of NETRAD w.r.t. RH")
plt.scatter(X_scaler.inverse_transform(X_uniform)[:, 2], uniform_preds[:, 0], c='r')

In [ ]:
plt.title("Partials of NETRAD w.r.t. RH")
plt.scatter(X_scaler.inverse_transform(X_uniform)[:, 2], uniform_partials[0][:, 2])

## Gaussian Predcitions/Partials

In [ ]:
plt.title("Predictions of NETRAD w.r.t. TA (Gaussian)")
plt.scatter(X_scaler.inverse_transform(X_gaussian)[:, 0], gauss_preds[:, 0], c='r')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#     ax.xlim([-20,20])
ax.scatter(X_scaler.inverse_transform(X_gaussian)[:, 0], X_scaler.inverse_transform(X_gaussian)[:, 1], gauss_preds[:, 0])

In [ ]:
plt.title("Partials of NETRAD w.r.t. TA")
plt.scatter(X_scaler.inverse_transform(X_gaussian)[:, 0], gauss_partials[0][:, 0])

In [ ]:
plt.title("Predictions of NETRAD w.r.t. SW_IN")
plt.scatter(X_scaler.inverse_transform(X_gaussian)[:, 1], gauss_preds[:, 0], c='r')

In [ ]:
plt.title("Partials of NETRAD w.r.t. SW_IN")
plt.scatter(X_scaler.inverse_transform(X_gaussian)[:, 1], gauss_partials[0][:, 1])

In [ ]:
plt.title("Predictions of NETRAD w.r.t. RH")
plt.scatter(X_scaler.inverse_transform(X_gaussian)[:, 2], gauss_preds[:, 0], c='r')

In [ ]:
plt.title("Partials of NETRAD w.r.t. RH")
plt.scatter(X_scaler.inverse_transform(X_gaussian)[:, 2], gauss_partials[0][:, 2])